In [1]:
import pymongo as pm
import urllib
from scrape import Scrape as sc
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re
import time

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

In [3]:
# define db and collections
db = client.scrape_db

# this collection will store all the HTML pages
htmlCollection = db.HTML_pages

In [4]:
startURL = 'http://books.toscrape.com/'
domain = 'toscrape.com'

done_urls = set()
external_urls = []

urls = set()
pagelinks = set()

urls.add(startURL)

In [5]:
ctr = 0
while (len(urls) > 0) and ctr < 3:    #(len(done_urls) < 200):
    
    forStart = time.time()
    for url in urls:
        print("URL in for loop: ", url)
        start = time.time()
        time.sleep(5)   # time governor
        html = sc.scrapeHTML(url, htmlCollection, done_urls)
        end = time.time()
        print(f'Elapsed time: {end - start}')
        if html: 
            #print(f'Html scraped: {html[:50]}')
            pagelinks = pagelinks.union(sc.scrapeURL(html, url))
            print(f'Extracted links: {len(pagelinks)}')
                  #{pagelinks}')
    forEnd = time.time()
    print()
    print(f'Number of Urls: {len(urls)}/nProcess Time: {forEnd -forStart}')
    print('--------------------------------------------------------------')
        
        
    
    # add links from page to urls set outside of for loop
    for link in pagelinks:
        if domain in link:
            urls.add(link)
            # print(f"Link in pagelinks: {link}")
        else:
            external_urls.add(link)

        
        
    
    print("Total urls: ", len(urls))
    
    # remove done_urls from todo urls
    urls -= done_urls
    print("done_urls: ", len(done_urls))
    print("(urls - done_urls): ", len(urls))
    ctr += 1
    print("LOOP counter: ", ctr)
    print("--------------------")


URL in for loop:  http://books.toscrape.com/
Elapsed time: 5.386628866195679
Extracted links: 73

Number of Urls: 1/nProcess Time: 5.408618688583374
--------------------------------------------------------------
Total urls:  74
done_urls:  1
(urls - done_urls):  73
LOOP counter:  1
--------------------
URL in for loop:  http://books.toscrape.com/catalogue/category/books/add-a-comment_18/index.html
Elapsed time: 6.966644525527954
Extracted links: 94
URL in for loop:  http://books.toscrape.com/catalogue/category/books/horror_31/index.html
Elapsed time: 8.187065839767456
Extracted links: 111
URL in for loop:  http://books.toscrape.com/catalogue/category/books/spirituality_39/index.html
Elapsed time: 5.211830139160156
Extracted links: 117
URL in for loop:  http://books.toscrape.com/catalogue/category/books/contemporary_38/index.html
Elapsed time: 5.206566572189331
Extracted links: 120
URL in for loop:  http://books.toscrape.com/catalogue/starving-hearts-triangular-trade-trilogy-1_990/index

Elapsed time: 7.5116119384765625
Extracted links: 454
URL in for loop:  http://books.toscrape.com/catalogue/category/books/food-and-drink_33/index.html
Elapsed time: 8.061274528503418
Extracted links: 474
URL in for loop:  http://books.toscrape.com/catalogue/category/books/christian_43/index.html
Elapsed time: 7.399579286575317
Extracted links: 477
URL in for loop:  http://books.toscrape.com/catalogue/category/books/cultural_49/index.html
Elapsed time: 5.239171743392944
Extracted links: 478
URL in for loop:  http://books.toscrape.com/catalogue/category/books/default_15/index.html
Elapsed time: 5.235859632492065
Extracted links: 492
URL in for loop:  http://books.toscrape.com/catalogue/category/books/mystery_3/index.html
Elapsed time: 5.21597695350647
Extracted links: 511
URL in for loop:  http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
Elapsed time: 7.945401668548584
Extracted links: 531
URL in for loop:  http://books.toscrape.com/catalogue/shakespear

Elapsed time: 5.211483716964722
Extracted links: 607
URL in for loop:  http://books.toscrape.com/catalogue/catastrophic-happiness-finding-joy-in-childhoods-messy-years_138/index.html
Elapsed time: 5.209999322891235
Extracted links: 607
URL in for loop:  http://books.toscrape.com/catalogue/take-me-home-tonight-rock-star-romance-3_605/index.html
Elapsed time: 5.211686849594116
Extracted links: 607
URL in for loop:  http://books.toscrape.com/catalogue/the-grand-design_405/index.html
Elapsed time: 5.21933650970459
Extracted links: 607
URL in for loop:  http://books.toscrape.com/catalogue/life_104/index.html
Elapsed time: 5.622184991836548
Extracted links: 607
URL in for loop:  http://books.toscrape.com/catalogue/the-great-railway-bazaar_446/index.html
Elapsed time: 7.855005264282227
Extracted links: 607
URL in for loop:  http://books.toscrape.com/catalogue/a-murder-in-time_877/index.html
Elapsed time: 6.221982717514038
Extracted links: 607
URL in for loop:  http://books.toscrape.com/catalo

Elapsed time: 5.213458776473999
Extracted links: 629
URL in for loop:  http://books.toscrape.com/catalogue/the-cookies-cups-cookbook-125-sweet-savory-recipes-reminding-you-to-always-eat-dessert-first_841/index.html
Elapsed time: 5.286261558532715
Extracted links: 629
URL in for loop:  http://books.toscrape.com/catalogue/murder-at-the-42nd-street-library-raymond-ambler-1_624/index.html
Elapsed time: 6.727445363998413
Extracted links: 629
URL in for loop:  http://books.toscrape.com/catalogue/angels-walking-angels-walking-1_662/index.html
Elapsed time: 7.856901168823242
Extracted links: 629
URL in for loop:  http://books.toscrape.com/catalogue/sophies-world_966/index.html
Elapsed time: 7.862200975418091
Extracted links: 629
URL in for loop:  http://books.toscrape.com/catalogue/modern-romance_820/index.html
Elapsed time: 5.549846410751343
Extracted links: 629
URL in for loop:  http://books.toscrape.com/catalogue/thomas-jefferson-and-the-tripoli-pirates-the-forgotten-war-that-changed-americ

Elapsed time: 6.809778690338135
Extracted links: 651
URL in for loop:  http://books.toscrape.com/catalogue/the-marriage-of-opposites_759/index.html
Elapsed time: 9.395735502243042
Extracted links: 651
URL in for loop:  http://books.toscrape.com/catalogue/category/books/food-and-drink_33/page-2.html
Elapsed time: 7.856910943984985
Extracted links: 662
URL in for loop:  http://books.toscrape.com/catalogue/the-road-to-little-dribbling-adventures-of-an-american-in-britain-notes-from-a-small-island-2_277/index.html
Elapsed time: 6.100650787353516
Extracted links: 662
URL in for loop:  http://books.toscrape.com/catalogue/naked_197/index.html
Elapsed time: 6.837992191314697
Extracted links: 662
URL in for loop:  http://books.toscrape.com/catalogue/the-new-drawing-on-the-right-side-of-the-brain_550/index.html
Elapsed time: 7.963843107223511
Extracted links: 662
URL in for loop:  http://books.toscrape.com/catalogue/sleeping-giants-themis-files-1_604/index.html
Elapsed time: 5.610799074172974
Ex

...

Elapsed time: 5.298148155212402
Extracted links: 709
URL in for loop:  http://books.toscrape.com/catalogue/the-last-girl-the-dominion-trilogy-1_70/index.html
Elapsed time: 7.9698052406311035
Extracted links: 709
URL in for loop:  http://books.toscrape.com/catalogue/overload-how-to-unplug-unwind-and-unleash-yourself-from-the-pressure-of-stress_725/index.html
Elapsed time: 5.297349452972412
Extracted links: 709
URL in for loop:  http://books.toscrape.com/catalogue/salt_731/index.html
Elapsed time: 6.943792343139648
Extracted links: 709
URL in for loop:  http://books.toscrape.com/catalogue/crazy-love-overwhelmed-by-a-relentless-god_682/index.html
Elapsed time: 7.8591673374176025
Extracted links: 709
URL in for loop:  http://books.toscrape.com/catalogue/lumberjanes-vol-1-beware-the-kitten-holy-lumberjanes-1-4_906/index.html
Elapsed time: 7.958298206329346
Extracted links: 709
URL in for loop:  http://books.toscrape.com/catalogue/worlds-elsewhere-journeys-around-shakespeares-globe_972/index

Elapsed time: 5.298396587371826
Extracted links: 754
URL in for loop:  http://books.toscrape.com/catalogue/twenties-girl_292/index.html
Elapsed time: 6.420027017593384
Extracted links: 754
URL in for loop:  http://books.toscrape.com/catalogue/the-argonauts_837/index.html
Elapsed time: 6.233656406402588
Extracted links: 754
URL in for loop:  http://books.toscrape.com/catalogue/higherselfie-wake-up-your-life-free-your-soul-find-your-tribe_957/index.html
Elapsed time: 5.3172948360443115
Extracted links: 754
URL in for loop:  http://books.toscrape.com/catalogue/my-name-is-lucy-barton_720/index.html
Elapsed time: 5.207097053527832
Extracted links: 754
URL in for loop:  http://books.toscrape.com/catalogue/the-song-of-achilles_762/index.html
Elapsed time: 5.321240663528442
Extracted links: 754
URL in for loop:  http://books.toscrape.com/catalogue/voyager-outlander-3_299/index.html
Elapsed time: 5.210674524307251
Extracted links: 754
URL in for loop:  http://books.toscrape.com/catalogue/i-am-a

Elapsed time: 5.589716672897339
Extracted links: 770
URL in for loop:  http://books.toscrape.com/catalogue/redeeming-love_826/index.html
Elapsed time: 5.295196294784546
Extracted links: 770
URL in for loop:  http://books.toscrape.com/catalogue/pet-sematary_726/index.html
Elapsed time: 5.21709680557251
Extracted links: 770
URL in for loop:  http://books.toscrape.com/catalogue/hollow-city-miss-peregrines-peculiar-children-2_813/index.html
Elapsed time: 5.2776172161102295
Extracted links: 770
URL in for loop:  http://books.toscrape.com/catalogue/the-travelers_285/index.html
Elapsed time: 7.751407623291016
Extracted links: 770
URL in for loop:  http://books.toscrape.com/catalogue/the-art-forger_930/index.html
Elapsed time: 5.2343056201934814
Extracted links: 770
URL in for loop:  http://books.toscrape.com/catalogue/the-kite-runner_847/index.html
Elapsed time: 6.281587362289429
Extracted links: 770
URL in for loop:  http://books.toscrape.com/catalogue/category/books/historical-fiction_4/pag

Elapsed time: 5.21895432472229
Extracted links: 777
URL in for loop:  http://books.toscrape.com/catalogue/everydata-the-misinformation-hidden-in-the-little-data-you-consume-every-day_891/index.html
Elapsed time: 6.906549692153931
Extracted links: 777
URL in for loop:  http://books.toscrape.com/catalogue/the-star-touched-queen_642/index.html
Elapsed time: 7.959674119949341
Extracted links: 777
URL in for loop:  http://books.toscrape.com/catalogue/burning_884/index.html
Elapsed time: 5.235191822052002
Extracted links: 777
URL in for loop:  http://books.toscrape.com/catalogue/32-yolks_510/index.html
Elapsed time: 7.172925710678101
Extracted links: 777
URL in for loop:  http://books.toscrape.com/catalogue/deliciously-ella-every-day-quick-and-easy-recipes-for-gluten-free-snacks-packed-lunches-and-simple-meals_801/index.html
Elapsed time: 5.277539491653442
Extracted links: 777
URL in for loop:  http://books.toscrape.com/catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html
Elapsed ti

Elapsed time: 7.524886846542358
Extracted links: 799
URL in for loop:  http://books.toscrape.com/catalogue/a-series-of-catastrophes-and-miracles-a-true-story-of-love-science-and-cancer_655/index.html
Elapsed time: 6.672747611999512
Extracted links: 799
URL in for loop:  http://books.toscrape.com/catalogue/zealot-the-life-and-times-of-jesus-of-nazareth_785/index.html
Elapsed time: 5.77860951423645
Extracted links: 799
URL in for loop:  http://books.toscrape.com/catalogue/the-art-book_490/index.html
Elapsed time: 5.208982944488525
Extracted links: 799
URL in for loop:  http://books.toscrape.com/catalogue/critique-of-pure-reason_366/index.html
Elapsed time: 6.220033168792725
Extracted links: 799
URL in for loop:  http://books.toscrape.com/catalogue/civilization-and-its-discontents_140/index.html
Elapsed time: 7.670668601989746
Extracted links: 799
URL in for loop:  http://books.toscrape.com/catalogue/girl-with-a-pearl-earring_322/index.html
Elapsed time: 7.855828762054443
Extracted links: